In [15]:
import numpy as np
import dataProsessing as dp
import matplotlib.pyplot as plt
import os
import io
import json
import math, random, time
from numpy import linalg as LA
import music21
from sklearn.cluster import KMeans
from tabulate import tabulate
%matplotlib inline


In [2]:
#RUN THIS ONCE
os.chdir("../")

Load corpi of chords:
Input needed: set model_name manually from available files in weights folder
- vocab: dictionary with key:value as (chord vector in midi format):index
- inv_vocab: chords as vector using midi numbers
- weights: weight vectors outputted from word2vec

inv_vocab and weights indexing are such that the ith entry in inv_vocab corresponds with the ith entry in weights

In [3]:
#set model_name manually
model_name = "d_128 n_15"

datasetName = "Jsb16thSeparated(t_60_rr_re)"

#add option for / vs \ check TODO
vocab = dataset = json.load(open("Data/Corpi/" + datasetName + "/" + datasetName +"_vocab.json"))
inv_vocab = vocab["inv_vocab"]
vocab = vocab["vocab"]
weights = json.load(open("Data/Corpi/" + datasetName + "/weights/" + model_name))
#weights_km = np.array(weights).T

Gather chord information and format ...

- chord_info: [ [chord name, chord root tone, chord quality] ] as determined by music21. 
- chord_names: [ chord name ] as determined by music21. 

In [6]:
chord_info = dp.getChordLabels(inv_vocab,True,True) #gets (chord name)

0th step
300th step
600th step
900th step
1200th step
1500th step
1800th step
2100th step
2400th step
2700th step
3000th step
3300th step
3600th step
3900th step
4200th step
4500th step
4800th step
5100th step
5400th step
5700th step
6000th step
6300th step
6600th step
6900th step


In [7]:
chord_names = chord_info[:,0]

Now we the sklearn KMeans implementation. Here we use 24 clusters.

In [28]:
k = 24
Kmean = KMeans(n_clusters=k).fit(weights)
Kmean.labels_

array([ 0,  9,  9, ..., 21, 15,  4], dtype=int32)

In [29]:
#Find the cluster centroids.
Centroids = Kmean.cluster_centers_
#print(Centroids)

Now we print a table containing some information about the clustering. Currently it is set to list the partitions with the top n most frequent chords. See the text files in OutputAnalysis folder for more examples and analysis.


In [30]:
namefreqs,rootfreqs,qualfreqs,partsizes =  dp.partitionFreqs(Kmean.labels_,chord_info,weights,k)
title,tabledata,headers = dp.tabulate_partitions("ChordNames",namefreqs,partsizes,5,k) #buggie doesn't handle well when n > #eles in partition
print(title)
print(tabulate(tabledata,headers))

Partitions, ChordNames
  Partition    part. size  #1 most common              #2 most common                     #3 most common                                   #4 most common                                #5 most common
-----------  ------------  --------------------------  ---------------------------------  -----------------------------------------------  --------------------------------------------  -------------------------------------------
          0        0.0779  ['C-major triad', 0.0255]   ['Eb-major triad', 0.0219]         ['F-major triad', 0.0219]                        ['C-minor triad', 0.02]                       ['F-minor triad', 0.0182]
          1        0.0253  ['G-major triad', 0.0449]   ['A-minor seventh chord', 0.0393]  ['C-major triad', 0.0337]                        ['A-minor triad', 0.0337]                     ['G-major-second major tetrachord', 0.0281]
          2        0.0434  ['G-major triad', 0.0327]   ['G-minor triad', 0.0294]          ['D-major triad', 

This will list the chords by cluster using chord names. 

In [17]:
#j will iterate through all the n clusters.
for j in range(1,16):
    #Create a list of chords in each cluster.
    chords_list = []
    for i in range(len(Kmean.labels_)):
        if Kmean.labels_[i] == j:
            chords_list.append(chord_names[i])
    if chords_list != []:
        print(f'The {j} cluster has these chords:', chords_list)
        
    

The 1 cluster has these chords: ['A-minor triad', 'D-minor triad', 'D-minor triad', 'C-major triad', 'C-minor triad', 'A-incomplete minor-seventh chord', 'C-major triad', 'D-dominant seventh chord', 'E-minor triad', 'C-minor triad', 'F-minor triad', 'Bb-minor triad', 'C-dominant seventh chord', 'F-minor triad', 'D-incomplete minor-seventh chord', 'A-minor triad', 'A-minor triad', 'E-minor triad', 'F-major triad', 'G-dominant seventh chord', 'E-major triad', 'C-minor triad', 'Ab-major seventh chord', 'F-minor seventh chord', 'A-half-diminished seventh chord', 'E#-diminished seventh chord', 'Eb-augmented triad', 'G-major triad', 'C-perfect-fourth minor tetrachord', 'F-dominant seventh chord', 'D-major triad', 'G-major-second minor tetrachord', 'Eb-major triad', 'A-half-diminished seventh chord', 'C-minor triad', 'D-half-diminished seventh chord', 'F-minor triad', 'G-dominant seventh chord', 'C-minor triad', 'C-major-second minor tetrachord', 'Major Third with octave doublings above G', '